<a href="https://colab.research.google.com/github/akib1162100/ML_base/blob/main/ML_intro_asg_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install numpy
!pip3 install pandas
!pip3 install keras
!pip3 install tensorflow

#Data Analysis and Preprocessing

In [19]:
import pandas as pd

ecommerce_raw = pd.read_excel('ecommerce_data.xlsx')

In [20]:
ecommerce_raw.head(10)

,CustomerID,Timestamp,ProductID,Category,Price,Quantity,Action
0,1052,2023-01-01,2,Clothing,125.570224,2,Add to Cart
1,1093,2023-01-02,15,Clothing,191.996781,3,Add to Cart
2,1015,2023-01-03,8,Clothing,40.645691,1,Add to Cart
3,1072,2023-01-04,8,NaN,NaN,2,View
4,1061,2023-01-05,17,NaN,NaN,5,Purchase
5,1021,2023-01-06,12,Books,369.471757,1,Add to Cart
6,1083,2023-01-07,19,Home,156.727320,1,View
7,1087,2023-01-08,10,Books,95.005236,2,Add to Cart
8,1075,2023-01-09,9,Electronics,322.401100,3,Purchase
9,1075,2023-01-10,5,Books,253.121528,1,View


**Cleaning Category** data and fill it with similar productID's category

In [21]:
ecommerce_raw.describe()

,CustomerID,ProductID,Price,Quantity
count,1000.000000,1000.000000,900.000000,1000.000000
mean,1050.128000,10.338000,257.705579,2.975000
std,29.573505,5.771921,144.306057,1.414346
min,1001.000000,1.000000,10.092316,1.000000
25%,1024.000000,5.000000,130.849891,2.000000
50%,1051.000000,10.000000,266.021046,3.000000
75%,1075.000000,15.000000,382.361857,4.000000
max,1100.000000,20.000000,499.859764,5.000000


In [22]:
ecommerce_raw['Category'].describe()

count          900
unique           4
top       Clothing
freq           239
Name: Category, dtype: object

In [23]:
product_category_map = ecommerce_raw.dropna(subset=['Category']).drop_duplicates(subset=['ProductID'])[['ProductID', 'Category']]
ecommerce_raw['Category'] = ecommerce_raw['Category'].fillna(ecommerce_raw['ProductID'].map(product_category_map.set_index('ProductID')['Category']))

In [24]:
ecommerce_raw['Category'].isna().any()

False

**Handeling Price data**

In [25]:
ecommerce_raw['Price'].describe()

count    900.000000
mean     257.705579
std      144.306057
min       10.092316
25%      130.849891
50%      266.021046
75%      382.361857
max      499.859764
Name: Price, dtype: float64

In [26]:
product_price_map = ecommerce_raw.dropna(subset=['Price']).drop_duplicates(subset=['ProductID'])[['ProductID', 'Price']]
ecommerce_raw['Price'] = ecommerce_raw['Price'].fillna(ecommerce_raw['ProductID'].map(product_price_map.set_index('ProductID')['Price']))

In [27]:
ecommerce_raw['Price'].isna().any()

False

Analyze customer interactions

In [36]:
len(ecommerce_raw['CustomerID'].unique())

100

In [67]:
customer_interaction_counts = ecommerce_raw.groupby('CustomerID')['Action'].count().reset_index()
customer_interaction_counts.rename(columns={'Action': 'TotalActions'}, inplace=True)
most_active_customer = customer_interaction_counts.loc[customer_interaction_counts['TotalActions'].idxmax()]

In [68]:
most_active_customer

CustomerID      1090
TotalActions      20
Name: 89, dtype: int64

#Feature Engineering and Analysis

Adding total cost column

In [69]:
ecommerce_raw['TotalSpent'] = ecommerce_raw["Price"]*ecommerce_raw["Quantity"]
ecommerce_raw['TotalSpent'].describe()

count    1000.000000
mean      780.415309
std       609.350831
min        16.031754
25%       270.824383
50%       628.384363
75%      1188.000571
max      2479.769412
Name: TotalSpent, dtype: float64

In [70]:
grouped = ecommerce_raw.groupby('CustomerID')
quantity_per_customer = grouped['Quantity'].sum()

In [71]:
cost_per_customer =  grouped['TotalSpent'].sum()
average_cost_per_customer = grouped['TotalSpent'].mean()

In [72]:
potential_valuable_customer = cost_per_customer.loc[cost_per_customer.idxmax()]
cost_per_customer.idxmax(),potential_valuable_customer

(1033, 18925.87592525672)

In [73]:
potential_loyal_valuable_per_customer = average_cost_per_customer.loc[average_cost_per_customer.idxmax()]
average_cost_per_customer.idxmax(),potential_loyal_valuable_per_customer

(1031, 1342.445784569562)

Filter By purchase action

In [74]:
only_purchase=ecommerce_raw[ecommerce_raw['Action'] == 'Purchase']
grouped_OP = only_purchase.groupby('CustomerID')

In [78]:
cost_per_customer_OP =  grouped_OP['TotalSpent'].sum()
average_cost_per_customer_OP = grouped_OP['TotalSpent'].mean()

In [79]:
valuable_customer_OP = cost_per_customer_OP.loc[cost_per_customer_OP.idxmax()]
cost_per_customer_OP.idxmax(),valuable_customer_OP

(1092, 12070.523164409899)

In [80]:
loyal_valuable_per_customer_OP = average_cost_per_customer_OP.loc[average_cost_per_customer_OP.idxmax()]
average_cost_per_customer_OP.idxmax(),loyal_valuable_per_customer_OP

(1055, 1759.5028763798998)

In [86]:
category_counts = ecommerce_raw['Category'].value_counts()
print(f"most interacted category is {category_counts.idxmax()} and it's value is {category_counts.loc[category_counts.idxmax()]}")

most interacted category is Clothing and it's value is 265


In [87]:
grouped_caegory = ecommerce_raw.groupby('Category')
price_category=grouped_caegory['Price'].sum()
print(f"most potential profitable category is {price_category.idxmax()} and it's value is {price_category.loc[price_category.idxmax()]}")

most potential profitable category is Electronics and it's value is 69575.62098416791


In [89]:
grouped_caegory_purchase = only_purchase.groupby('Category')
price_category_purchased=grouped_caegory_purchase['Price'].sum()
print(f"most pricy category is {price_category_purchased.idxmax()} and it's value is {price_category_purchased.loc[price_category_purchased.idxmax()]}")

most pricy category is Electronics and it's value is 64283.52246876539
